In [72]:
import nltk
import shutil

In [33]:
%run dataset.ipynb
%run model.ipynb

In [79]:
batch_size = 20
input_folder = './sample test/'
output_folder = './sample summaries/'
filenames = [f for f in os.listdir(folder) if ".txt" in f]

In [17]:
g = GloveEmbeddings()
g.load_dump('../data/glove.pkl')

In [58]:
m = EncoderDecoder(torch.cuda.FloatTensor(g.vectors), word_emb_size=g.dim, 
                       sen_emb_size=350, doc_emb_size=600, sen_len=50, batch_size=20, output_dim=2,
                       reverse=True)

m.load_state_dict(torch.load("./cnn.params"))
m.cuda()

EncoderDecoder(
  (sentenceEncoder): KimCNN(
    (embeddings): Embedding(400002, 200)
    (convolutions): ModuleList(
      (0): Conv2d(1, 50, kernel_size=(1, 200), stride=(1, 1))
      (1): Conv2d(1, 50, kernel_size=(2, 200), stride=(1, 1))
      (2): Conv2d(1, 50, kernel_size=(3, 200), stride=(1, 1))
      (3): Conv2d(1, 50, kernel_size=(4, 200), stride=(1, 1))
      (4): Conv2d(1, 50, kernel_size=(5, 200), stride=(1, 1))
      (5): Conv2d(1, 50, kernel_size=(6, 200), stride=(1, 1))
      (6): Conv2d(1, 50, kernel_size=(7, 200), stride=(1, 1))
    )
  )
  (documentEncoder): LSTM(350, 600, batch_first=True)
  (documentDecoder): LSTM(350, 600, batch_first=True)
  (classifier): Linear(in_features=600, out_features=2, bias=True)
)

In [68]:
for i in range(0, len(filenames), batch_size):
    summaries = []

    for filename in filenames[i:i+batch_size]:
        d.data = []

        text = open(os.path.join(folder, filename)).read()
        sentences = nltk.tokenize.sent_tokenize(text)
        words = [nltk.tokenize.word_tokenize(sentence) for sentence in sentences]
        d.data.append(g.convert_to_indices(words))
        
        d.pad()

        with torch.no_grad():
            pred = m(torch.cuda.LongTensor(d.lines))
            for i, lines in enumerate(pred.data):
                pos = [l[1].data for l in lines]
                selected_lines = list(zip(*sorted(zip(pos, range(len(pos))), key=lambda x: x[0], reverse=True)))[1][0:summary_len]
                selected_lines = [sentences[i] for i in selected_lines if i < len(words[i])]
                summaries.append(selected_lines)

In [80]:
if os.path.exists(output_folder): shutil.rmtree(output_folder)
os.mkdir(output_folder)

In [78]:
for filename, summary in zip(filenames, summaries):
    with open(os.path.join(output_folder, filename), 'w+') as f:
        f.writelines(summary)